Import libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [32]:
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import r2_score, mean_squared_error

In [33]:
import pickle

In [31]:
import warnings
warnings.filterwarnings('once')

---

# Prepare dataset

## Load data

In [3]:
df = pd.read_csv('homework/radar_parameters.csv', index_col=0)
df

,Zh (dBZ),Zdr (dB),Ldr (dB),Kdp (deg km-1),Ah (dBZ/km),Adr (dB/km),R (mm/hr)
0,23.144878,0.418637,-41.757733,0.005395,0.000290,0.000012,2.393520
1,22.737156,0.322850,-43.772069,0.005194,0.000360,0.000012,3.502699
2,26.869826,0.330948,-43.577399,0.013385,0.000903,0.000030,8.627561
3,28.540561,0.399480,-42.139731,0.018872,0.001036,0.000043,8.424447
4,30.500127,0.543758,-39.763087,0.027438,0.001157,0.000064,8.189291
...,...,...,...,...,...,...,...
18964,31.515997,0.579955,-39.244229,0.034048,0.001417,0.000080,10.648020
18965,29.993334,0.567935,-39.399188,0.024134,0.001032,0.000057,7.981875
18966,31.685913,0.655681,-38.375696,0.033971,0.001165,0.000081,6.822691
18967,32.980096,0.768586,-37.166218,0.043117,0.001285,0.000105,6.801169


## Split dataset

In [8]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

In [9]:
X_train, X_test, y_train, y_test = \
    train_test_split(
        X, y, 
        random_state=42,
        train_size=0.7)

---

# Multiple linear regression

In [13]:
model1 = LinearRegression(fit_intercept=True)

%time model1.fit(X_train, y_train)

CPU times: user 71.5 ms, sys: 34.3 ms, total: 106 ms
Wall time: 60.2 ms


LinearRegression()

In [16]:
y_pred = model1.predict(X_test)

print('R^2 score:', r2_score(y_test, y_pred))
print('RMSE score:', mean_squared_error(y_test, y_pred))

R^2 score: 0.9890992951689395
RMSE score: 0.8757449868846127


For baseline relationship $Z=200R^{1.6}$:

In [20]:
def R_empirical(Zh):
    Z = 10**(Zh/10)
    R = (Z/200)**(1/1.6)
    return R

print('R^2 score:', r2_score(y, R_empirical(X['Zh (dBZ)'])))
print('RMSE score:', mean_squared_error(y, R_empirical(X['Zh (dBZ)'])))

R^2 score: 0.3023229070437503
RMSE score: 51.23110663345855


In [34]:
with open('model1.pkl','wb') as f:
    pickle.dump(model1,f)

# Polynominal regression

In [21]:
np.arange(21)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20])

In [43]:
def PolynomialRegression(degree=2, **kwargs):
    return make_pipeline(PolynomialFeatures(degree),
                         LinearRegression(**kwargs))

param_grid = {'polynomialfeatures__degree': np.arange(21),
              'linearregression__fit_intercept': [True, False],
              'linearregression__normalize': [True, False]}

# grid1 = GridSearchCV(
#     PolynomialRegression(verbose=True, n_jobs=-1), 
#     param_grid, cv=7, n_jobs=-1)
grid1 = RandomizedSearchCV(
    RandomForestRegressor(n_jobs=-1), 
    param_distribution, 
    n_iter=20, cv=5, verbose=10)

In [44]:
%%time
grid1.fit(X_train, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV 1/5; 1/20] START bootstrap=True, max_depth=90, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=1600
[CV 1/5; 1/20] END bootstrap=True, max_depth=90, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=1600;, score=0.983 total time=   7.4s
[CV 2/5; 1/20] START bootstrap=True, max_depth=90, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=1600
[CV 2/5; 1/20] END bootstrap=True, max_depth=90, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=1600;, score=0.986 total time=   5.0s
[CV 3/5; 1/20] START bootstrap=True, max_depth=90, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=1600
[CV 3/5; 1/20] END bootstrap=True, max_depth=90, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=1600;, score=0.959 total time=   4.9s
[CV 4/5; 1/20] START bootstrap=True, max_depth=90, max_features=sqrt, m

/opt/anaconda3/envs/default/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


[CV 1/5; 3/20] END bootstrap=True, max_depth=60, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=1200;, score=0.987 total time=   9.2s
[CV 2/5; 3/20] START bootstrap=True, max_depth=60, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=1200


/opt/anaconda3/envs/default/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


[CV 2/5; 3/20] END bootstrap=True, max_depth=60, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=1200;, score=0.989 total time=   9.1s
[CV 3/5; 3/20] START bootstrap=True, max_depth=60, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=1200


...

/opt/anaconda3/envs/default/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


[CV 5/5; 19/20] END bootstrap=False, max_depth=40, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=1200;, score=0.972 total time=  13.7s
[CV 1/5; 20/20] START bootstrap=False, max_depth=50, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=400
[CV 1/5; 20/20] END bootstrap=False, max_depth=50, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=400;, score=0.985 total time=   3.0s
[CV 2/5; 20/20] START bootstrap=False, max_depth=50, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=400
[CV 2/5; 20/20] END bootstrap=False, max_depth=50, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=400;, score=0.986 total time=   3.4s
[CV 3/5; 20/20] START bootstrap=False, max_depth=50, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=400
[CV 3/5; 20/20] END bootstrap=False, max_depth=50, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=40

/opt/anaconda3/envs/default/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


CPU times: user 3min 43s, sys: 1min 16s, total: 5min
Wall time: 9min 3s


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(n_jobs=-1), n_iter=20,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   verbose=10)

In [27]:
grid.best_params_

{'linearregression__fit_intercept': False,
 'linearregression__normalize': True,
 'polynomialfeatures__degree': 2}

In [28]:
model2 = grid.best_estimator_
y_pred = model2.predict(X_test)

print('R^2 score:', r2_score(y_test, y_pred))
print('RMSE score:', mean_squared_error(y_test, y_pred))

R^2 score: 0.9995805761885463
RMSE score: 0.033695830311260035


In [35]:
with open('grid1.pkl','wb') as f:
    pickle.dump(grid,f)

---

# Random Forest Regressor

In [46]:
param_distribution = {
    'bootstrap': [True, False],  
    'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],  
    'max_features': ['auto', 'sqrt'],  
    'min_samples_leaf': [1, 2, 4],  
    'min_samples_split': [2, 5, 10],  
    'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}

grid2 = RandomizedSearchCV(
    RandomForestRegressor(n_jobs=-1), 
    param_distribution, 
    n_iter=40, cv=5, verbose=10)

In [47]:
%%time
grid2.fit(X_train, y_train)

Fitting 5 folds for each of 40 candidates, totalling 200 fits
[CV 1/5; 1/40] START bootstrap=False, max_depth=90, max_features=auto, min_samples_leaf=2, min_samples_split=10, n_estimators=1800


/opt/anaconda3/envs/default/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


[CV 1/5; 1/40] END bootstrap=False, max_depth=90, max_features=auto, min_samples_leaf=2, min_samples_split=10, n_estimators=1800;, score=0.976 total time=  18.2s
[CV 2/5; 1/40] START bootstrap=False, max_depth=90, max_features=auto, min_samples_leaf=2, min_samples_split=10, n_estimators=1800


/opt/anaconda3/envs/default/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


[CV 2/5; 1/40] END bootstrap=False, max_depth=90, max_features=auto, min_samples_leaf=2, min_samples_split=10, n_estimators=1800;, score=0.978 total time=  16.0s
[CV 3/5; 1/40] START bootstrap=False, max_depth=90, max_features=auto, min_samples_leaf=2, min_samples_split=10, n_estimators=1800


...

/opt/anaconda3/envs/default/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


[CV 5/5; 39/40] END bootstrap=False, max_depth=20, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=600;, score=0.972 total time=   7.3s
[CV 1/5; 40/40] START bootstrap=False, max_depth=40, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=1600
[CV 1/5; 40/40] END bootstrap=False, max_depth=40, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=1600;, score=0.983 total time=   7.6s
[CV 2/5; 40/40] START bootstrap=False, max_depth=40, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=1600
[CV 2/5; 40/40] END bootstrap=False, max_depth=40, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=1600;, score=0.984 total time=   7.5s
[CV 3/5; 40/40] START bootstrap=False, max_depth=40, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=1600
[CV 3/5; 40/40] END bootstrap=False, max_depth=40, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_est

/opt/anaconda3/envs/default/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


CPU times: user 6min 28s, sys: 2min 28s, total: 8min 57s
Wall time: 26min 10s


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(n_jobs=-1), n_iter=40,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   verbose=10)

In [49]:
grid2.best_params_

{'n_estimators': 600,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'auto',
 'max_depth': 60,
 'bootstrap': True}

In [50]:
model3 = grid2.best_estimator_
y_pred = model3.predict(X_test)

print('R^2 score:', r2_score(y_test, y_pred))
print('RMSE score:', mean_squared_error(y_test, y_pred))

R^2 score: 0.9874465602550847
RMSE score: 1.0085230354501793


In [48]:
with open('grid2.pkl','wb') as f:
    pickle.dump(grid2,f)

# Summary

Performance rank in both $R^2$ and root mean square error: 

||  $R^2$  | RMSE |
|---|---|---|
|1. Polynomial regression (degree 2) | 0.9995805761885463 | 0.033695830311260035 |
|2. Linear regression | 0.9890992951689395 | 0.8757449868846127 |
|3. RandomTree* | 0.9874465602550847 |1.0085230354501793 |
|4. The baseline prediction model | 0.3023229070437503 | 51.23110663345855 |

*Best RandomTree estimator: 

    ```python
    {'n_estimators': 600,
     'min_samples_split': 2,
     'min_samples_leaf': 1,
     'max_features': 'auto',
     'max_depth': 60,
     'bootstrap': True}
    ```
